# Hyperparameter Tuning using HyperDrive

TODO: Import Dependencies. In the cell below, import all the dependencies that you will need to complete the project.

## Create an experiment in workspace

In [7]:
from azureml.core import Workspace, Experiment
#Define a workspace
ws = Workspace.from_config()

#Create an experiment
exp = Experiment(workspace=ws, name="Hyperparameter_Tuning")
print('Workspace name: ' + ws.name, 
      'Azure region: ' + ws.location, 
      'Subscription id: ' + ws.subscription_id, 
      'Resource group: ' + ws.resource_group, sep = '\n')
run = exp.start_logging()


Workspace name: quick-starts-ws-243949
Azure region: westeurope
Subscription id: 81cefad3-d2c9-4f77-a466-99a7f541c7bb
Resource group: aml-quickstarts-243949


## Create cluster

In [8]:
from azureml.core.compute import ComputeTarget, AmlCompute
from azureml.core.compute_target import ComputeTargetException

cpu_cluster_name = "AzureProjecthp"

# TODO: Create compute cluster
# Use vm_size = " Standard_D2_V2"in your provisioning configuration.
# max_nodes should be no greater than 4.

### YOUR CODE HERE ###
# verify that cluser does not exist already
try:
    cpu_cluster=ComputeTarget(workspace=ws, name= cpu_cluster_name)
    print("Found existing cluster, use it.")
except ComputeTargetException:
    compute_config=AmlCompute.provisioning_configuration(vm_size="Standard_D2_V2", max_nodes=4)
    cpu_cluster= ComputeTarget.create(ws,cpu_cluster_name,compute_config)

cpu_cluster.wait_for_completion(show_output=True)
# print detailed status for the current cluster
print(cpu_cluster.get_status().serialize())

Found existing cluster, use it.
Succeeded
AmlCompute wait for completion finished

Minimum number of nodes requested have been provisioned
{'currentNodeCount': 0, 'targetNodeCount': 4, 'nodeStateCounts': {'preparingNodeCount': 0, 'runningNodeCount': 0, 'idleNodeCount': 0, 'unusableNodeCount': 0, 'leavingNodeCount': 0, 'preemptedNodeCount': 0}, 'allocationState': 'Resizing', 'allocationStateTransitionTime': '2023-11-11T17:19:37.215000+00:00', 'errors': None, 'creationTime': '2023-11-11T17:07:22.357693+00:00', 'modifiedTime': '2023-11-11T17:07:32.364898+00:00', 'provisioningState': 'Succeeded', 'provisioningStateTransitionTime': None, 'scaleSettings': {'minNodeCount': 0, 'maxNodeCount': 4, 'nodeIdleTimeBeforeScaleDown': 'PT1800S'}, 'vmPriority': 'Dedicated', 'vmSize': 'STANDARD_D2_V2'}


## Dataset

TODO: Get data. In the cell below, write code to access the data you will be using in this project. Remember that the dataset needs to be external.

In [9]:
import pandas as pd
pd_path='https://raw.githubusercontent.com/AnnaDM87/Udacity_CAPSTONE/main/starter_file/Employee.csv'
df = pd.read_csv(pd_path)

df.head()

,Education,JoiningYear,City,PaymentTier,Age,Gender,EverBenched,ExperienceInCurrentDomain,LeaveOrNot
0,Bachelors,2017,Bangalore,3,34,Male,No,0,0
1,Bachelors,2013,Pune,1,28,Female,No,3,1
2,Bachelors,2014,New Delhi,3,38,Female,No,2,0
3,Masters,2016,Bangalore,3,27,Male,No,5,1
4,Masters,2017,Pune,3,24,Male,Yes,2,1


In [10]:
df.isna().sum()

Education                    0
JoiningYear                  0
City                         0
PaymentTier                  0
Age                          0
Gender                       0
EverBenched                  0
ExperienceInCurrentDomain    0
LeaveOrNot                   0
dtype: int64

## Hyperdrive Configuration

TODO: Explain the model you are using and the reason for chosing the different hyperparameters, termination policy and config settings.

In [13]:
from azureml.widgets import RunDetails
from azureml.train.sklearn import SKLearn
from azureml.train.hyperdrive.run import PrimaryMetricGoal
from azureml.train.hyperdrive.policy import BanditPolicy
from azureml.train.hyperdrive.sampling import RandomParameterSampling
from azureml.train.hyperdrive.runconfig import HyperDriveConfig
from azureml.train.hyperdrive.parameter_expressions import choice, uniform
from azureml.core import Environment, ScriptRunConfig
import os

# Specify parameter sampler
ps = RandomParameterSampling(
{'--C':choice(0.01,0.1,1,10,50,100,500,1000),
'--max_iter':choice(50,100,200,300)})

# Specify a Policy
policy = BanditPolicy(evaluation_interval=1,slack_factor=0.1)

if "training" not in os.listdir():
    os.mkdir("./training")

# Setup environment for your training run
sklearn_env = Environment.from_conda_specification(name='sklearn-env', file_path='conda_dependencies.yml')

# Create a ScriptRunConfig Object to specify the configuration details of your training job
src = ScriptRunConfig(source_directory='.',
                     script='train.py',
                     compute_target=cpu_cluster,
                     environment=sklearn_env)

# Create a HyperDriveConfig using the src object, hyperparameter sampler, and policy.
hyperdrive_config = HyperDriveConfig(run_config=src,
                                    hyperparameter_sampling=ps,
                                    policy=policy,
                                    primary_metric_name='Accuracy',
                                    primary_metric_goal=PrimaryMetricGoal.MAXIMIZE,
                                    max_total_runs=12
                                    max_concurrent_runs=3)

SyntaxError: invalid syntax (3382046545.py, line 38)

In [ ]:
#TODO: Submit your experiment
# Submit your hyperdrive run to the experiment and show run details with the widget.

hyperdrive_run= exp.submit(hyperdrive_config,show_output=True)
RunDetails(hyperdrive_run).show()

hyperdrive_run.get_status()
hyperdrive_run.wait_for_completion(show_output=True)

_HyperDriveWidget(widget_settings={'childWidgetDisplay': 'popup', 'send_telemetry': False, 'log_level': 'INFO'…

RunId: HD_825afaca-ac9e-436b-9ab4-41bc15ff3e83
Web View: https://ml.azure.com/runs/HD_825afaca-ac9e-436b-9ab4-41bc15ff3e83?wsid=/subscriptions/81cefad3-d2c9-4f77-a466-99a7f541c7bb/resourcegroups/aml-quickstarts-243949/workspaces/quick-starts-ws-243949&tid=660b3398-b80e-49d2-bc5b-ac1dc93b5254

Streaming azureml-logs/hyperdrive.txt

[2023-11-11T17:24:44.847629][GENERATOR][INFO]Trying to sample '12' jobs from the hyperparameter space
[2023-11-11T17:24:45.5122798Z][SCHEDULER][INFO]Scheduling job, id='HD_825afaca-ac9e-436b-9ab4-41bc15ff3e83_0' 
[2023-11-11T17:24:45.6398752Z][SCHEDULER][INFO]Scheduling job, id='HD_825afaca-ac9e-436b-9ab4-41bc15ff3e83_1' 
[2023-11-11T17:24:45.7876885Z][SCHEDULER][INFO]Scheduling job, id='HD_825afaca-ac9e-436b-9ab4-41bc15ff3e83_2' 
[2023-11-11T17:24:45.8609859Z][SCHEDULER][INFO]Successfully scheduled a job. Id='HD_825afaca-ac9e-436b-9ab4-41bc15ff3e83_0' 
[2023-11-11T17:24:46.0189784Z][SCHEDULER][INFO]Scheduling job, id='HD_825afaca-ac9e-436b-9ab4-41bc15ff3e83_

## Run Details

OPTIONAL: Write about the different models trained and their performance. Why do you think some models did better than others?

TODO: In the cell below, use the `RunDetails` widget to show the different experiments.

## Best Model

TODO: In the cell below, get the best model from the hyperdrive experiments and display all the properties of the model.

In [24]:
import joblib
# Get your best run and save the model from that run.
print(hyperdrive_run.get_children_sorted_by_primary_metric(top=0,reverse =False, discard_no_metric=False))
### YOUR CODE HERE ###
best_run=hyperdrive_run.get_best_run_by_primary_metric()
print("Best run metric:", best_run.get_metrics())
print("Run details",best_run.get_details() )
print("Best run file names", best_run.get_file_names())

[{'run_id': 'HD_a01f5327-9fdd-4520-a82f-410355d0feff_9', 'hyperparameters': '{"--C": 500, "--max_iter": 100}', 'best_primary_metric': 0.7218045112781954, 'status': 'Completed'}, {'run_id': 'HD_a01f5327-9fdd-4520-a82f-410355d0feff_4', 'hyperparameters': '{"--C": 100, "--max_iter": 200}', 'best_primary_metric': 0.719656283566058, 'status': 'Completed'}, {'run_id': 'HD_a01f5327-9fdd-4520-a82f-410355d0feff_5', 'hyperparameters': '{"--C": 100, "--max_iter": 300}', 'best_primary_metric': 0.719656283566058, 'status': 'Completed'}, {'run_id': 'HD_a01f5327-9fdd-4520-a82f-410355d0feff_7', 'hyperparameters': '{"--C": 0.1, "--max_iter": 300}', 'best_primary_metric': 0.719656283566058, 'status': 'Completed'}, {'run_id': 'HD_a01f5327-9fdd-4520-a82f-410355d0feff_2', 'hyperparameters': '{"--C": 10, "--max_iter": 200}', 'best_primary_metric': 0.7175080558539205, 'status': 'Completed'}, {'run_id': 'HD_a01f5327-9fdd-4520-a82f-410355d0feff_6', 'hyperparameters': '{"--C": 10, "--max_iter": 100}', 'best_pri

In [25]:
best_run.get_file_names()

['outputs/model.joblib',
 'system_logs/cs_capability/cs-capability.log',
 'system_logs/hosttools_capability/hosttools-capability.log',
 'system_logs/lifecycler/execution-wrapper.log',
 'system_logs/lifecycler/lifecycler.log',
 'system_logs/metrics_capability/metrics-capability.log',
 'system_logs/snapshot_capability/snapshot-capability.log',
 'user_logs/std_log.txt']

In [26]:
hypr_mdl='hyperdrive_model'
folder='outputs'
path=os.path.join(folder,'model.joblib')
hyperdrive_model=best_run.register_model(model_name=hypr_mdl, model_path=path)

## Model Deployment

Remember you have to deploy only one of the two models you trained but you still need to register both the models. Perform the steps in the rest of this notebook only if you wish to deploy this model.

TODO: In the cell below, register the model, create an inference config and deploy the model as a web service.

TODO: In the cell below, send a request to the web service you deployed to test it.

TODO: In the cell below, print the logs of the web service and delete the service

**Submission Checklist**
- I have registered the model.
- I have deployed the model with the best accuracy as a webservice.
- I have tested the webservice by sending a request to the model endpoint.
- I have deleted the webservice and shutdown all the computes that I have used.
- I have taken a screenshot showing the model endpoint as active.
- The project includes a file containing the environment details.

